In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/completed_orders.csv')
delivery_requests = pd.read_csv('../data/drivers_location_during_request.csv')

In [3]:
## Feature Engineering
# Extract date components from timestamps

# Convert `Trip Start Time` and `Trip End Time` to datetime
df['Trip Start Time'] = pd.to_datetime(df['Trip Start Time'])
df['Trip End Time'] = pd.to_datetime(df['Trip End Time'])

# Impute missing values in `Trip Start Time` with the median
median_start_time = df['Trip Start Time'].median()
df['Trip Start Time'] = df['Trip Start Time'].fillna(median_start_time)

# Extract `day_of_week`, `hour_of_day`, `day_of_month`, and `month` from `Trip Start Time`
df['day_of_week'] = df['Trip Start Time'].dt.day_name()
df['hour_of_day'] = df['Trip Start Time'].dt.hour
df['day_of_month'] = df['Trip Start Time'].dt.day
df['month'] = df['Trip Start Time'].dt.month_name()

# Extract `latitude` and `longitude` from `Trip Origin` and `Trip Destination`
for col in ['Trip Origin', 'Trip Destination']:
    df[[f'{col}_latitude', f'{col}_longitude']] = df[col].str.split(',', expand=True).astype(float)

# Calculate `trip_duration` in seconds
df['trip_duration'] = (df['Trip End Time'] - df['Trip Start Time']).dt.total_seconds()

# Display the first 5 rows
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))


| Trip ID   | Trip Origin                        | Trip Destination                | Trip Start Time     | Trip End Time       | day_of_week   | hour_of_day   | day_of_month   | month   | Trip Origin_latitude   | Trip Origin_longitude   | Trip Destination_latitude   | Trip Destination_longitude   | trip_duration   |
|:----------|:-----------------------------------|:--------------------------------|:--------------------|:--------------------|:--------------|:--------------|:---------------|:--------|:-----------------------|:------------------------|:----------------------------|:-----------------------------|:----------------|
| 391996    | 6.508813001668548,3.37740316890347 | 6.650969799999999,3.3450307     | 2021-07-01 07:28:04 | 2021-07-01 07:29:37 | Thursday      | 7             | 1              | July    | 6.50881                | 3.3774                  | 6.65097                     | 3.34503                      | 93              |
| 391997    | 6.4316714,3.4555375             

In [4]:
df.tail()

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,day_of_week,hour_of_day,day_of_month,month,Trip Origin_latitude,Trip Origin_longitude,Trip Destination_latitude,Trip Destination_longitude,trip_duration
536015,1637696,"6.448218499999999,3.4772075","6.437787399999999,3.481670199999999",2021-12-30 20:35:06,2021-12-30 21:02:59,Thursday,20,30,December,6.448218,3.477208,6.437787,3.481670,1673.0
536016,1637702,"6.442320899999999,3.4736868","6.436589333407897,3.5559738188407835",2021-12-30 20:48:13,2021-12-30 21:43:49,Thursday,20,30,December,6.442321,3.473687,6.436589,3.555974,3336.0
536017,1637704,"6.4281982,3.492248","6.448088500000001,3.4775747",2021-12-30 20:51:45,2021-12-30 21:41:32,Thursday,20,30,December,6.428198,3.492248,6.448089,3.477575,2987.0
536018,1637705,"6.5869296,3.3632966","6.637906899999999,3.3339515",2021-12-30 20:48:50,2021-12-30 21:08:28,Thursday,20,30,December,6.586930,3.363297,6.637907,3.333951,1178.0
536019,1637709,"6.647209999999999,3.4851489","6.454915198823159,3.555839938365194",2021-12-30 20:55:38,2021-12-30 22:25:00,Thursday,20,30,December,6.647210,3.485149,6.454915,3.555840,5362.0


In [5]:
import altair as alt

# Filter out negative trip durations
df_filtered = df[df['trip_duration'] > 0].copy()

# Filter out trip durations exceeding three standard deviations from the mean
mean_trip_duration = df_filtered['trip_duration'].mean()
std_trip_duration = df_filtered['trip_duration'].std()
df_filtered = df_filtered[df_filtered['trip_duration'] <= mean_trip_duration + 3 * std_trip_duration]

# Recalculate and print summary statistics for `trip_duration`
print("Filtered Trip Duration Summary Statistics:")
print(df_filtered['trip_duration'].describe().to_markdown(numalign="left", stralign="left"))

# Scatter plot of Trip Origins colored by hour_of_day
chart1 = alt.Chart(df_filtered).mark_circle(size=60).encode(
    x=alt.X('Trip Origin_longitude:Q', title='Longitude'),
    y=alt.Y('Trip Origin_latitude:Q', title='Latitude'),
    color=alt.Color('hour_of_day:O', scale=alt.Scale(scheme='category20'), legend=alt.Legend(title='Hour of Day')),
    tooltip=['Trip Origin_latitude:Q', 'Trip Origin_longitude:Q', 'hour_of_day:O']
).properties(
    title='Trip Origins Colored by Hour of Day'
).interactive()

chart1.save('trip_origins_colored_by_hour.json')

# Scatter plot of Trip Destinations colored by hour_of_day
chart2 = alt.Chart(df_filtered).mark_circle(size=60).encode(
    x=alt.X('Trip Destination_longitude:Q', title='Longitude'),
    y=alt.Y('Trip Destination_latitude:Q', title='Latitude'),
    color=alt.Color('hour_of_day:O', scale=alt.Scale(scheme='category20'), legend=alt.Legend(title='Hour of Day')),
    tooltip=['Trip Destination_latitude:Q', 'Trip Destination_longitude:Q', 'hour_of_day:O']
).properties(
    title='Trip Destinations Colored by Hour of Day'
).interactive()

chart2.save('trip_destinations_colored_by_hour.json')

# 2D histogram of Trip Origins
chart3 = alt.Chart(df_filtered).mark_rect().encode(
    x=alt.X('Trip Origin_longitude:Q', bin=True, title='Longitude'),
    y=alt.Y('Trip Origin_latitude:Q', bin=True, title='Latitude'),
    color=alt.Color('count()', title='Density'),
    tooltip=[alt.Tooltip('Trip Origin_longitude:Q', bin=True, title='Longitude'), alt.Tooltip('Trip Origin_latitude:Q', bin=True, title='Latitude'), 'count()']
).properties(
    title='Density of Trip Origins'
).interactive()

chart3.save('trip_origins_density.json')

# 2D histogram of Trip Destinations
chart4 = alt.Chart(df_filtered).mark_rect().encode(
    x=alt.X('Trip Destination_longitude:Q', bin=True, title='Longitude'),
    y=alt.Y('Trip Destination_latitude:Q', bin=True, title='Latitude'),
    color=alt.Color('count()', title='Density'),
    tooltip=[alt.Tooltip('Trip Destination_longitude:Q', bin=True, title='Longitude'), alt.Tooltip('Trip Destination_latitude:Q', bin=True, title='Latitude'), 'count()']
).properties(
    title='Density of Trip Destinations'
).interactive()

chart4.save('trip_destinations_density.json')

Filtered Trip Duration Summary Statistics:
|       | trip_duration   |
|:------|:----------------|
| count | 534324          |
| mean  | 4555.05         |
| std   | 11517.9         |
| min   | 1               |
| 25%   | 2021            |
| 50%   | 3177            |
| 75%   | 4807            |
| max   | 794381          |


In [7]:
from altair_saver import save

# Save the chart as a PNG image
save(chart1, 'chart1.png')

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000).

Try enabling the VegaFusion data transformer which raises this limit by pre-evaluating data
transformations in Python.
    >> import altair as alt
    >> alt.data_transformers.enable("vegafusion")

Or, see https://altair-viz.github.io/user_guide/large_datasets.html for additional information
on how to plot large datasets.

In [8]:
from altair_saver import save

# Save the chart as a PNG image
save(chart1, 'chart1.png')


MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000).

Try enabling the VegaFusion data transformer which raises this limit by pre-evaluating data
transformations in Python.
    >> import altair as alt
    >> alt.data_transformers.enable("vegafusion")

Or, see https://altair-viz.github.io/user_guide/large_datasets.html for additional information
on how to plot large datasets.

In [9]:
from IPython.display import Image

# Display the saved image
Image(filename='chart1.png')

FileNotFoundError: [Errno 2] No such file or directory: 'chart1.png'

#### Get public holidays and weekends

In [13]:
# Convert 'Trip Start Time' to date only (no time component)
df['Trip Start Date'] = pd.to_datetime(df['Trip Start Time']).dt.date

# Get public holidays for Nigeria (replace with appropriate API/data source if needed)
def get_public_holidays(year):
    url = f"https://date.nager.at/api/v3/PublicHolidays/{year}/NG"
    response = requests.get(url)
    if response.status_code == 200:
        return [datetime.strptime(holiday['date'], '%Y-%m-%d').date() for holiday in response.json()]
    else:
        return []

holidays_2021 = get_public_holidays(2021)

df['is_holiday'] = df['Trip Start Date'].isin(holidays_2021).astype(int)

# 2. Weekend vs. Weekday
df['is_weekend'] = pd.to_datetime(df['Trip Start Time']).dt.dayofweek.isin([5, 6]).astype(int)

In [15]:
# save df to csv
df.to_csv('../data/completed_orders_with_features.csv', index=False)

In [14]:
# Display the first 5 rows
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

| Trip ID   | Trip Origin                        | Trip Destination                | Trip Start Time   | Trip End Time       | day_of_week   | hour_of_day   | day_of_month   | month   | Trip Origin_latitude   | Trip Origin_longitude   | Trip Destination_latitude   | Trip Destination_longitude   | trip_duration   | is_holiday   | is_weekend   | Trip Start Date   |
|:----------|:-----------------------------------|:--------------------------------|:------------------|:--------------------|:--------------|:--------------|:---------------|:--------|:-----------------------|:------------------------|:----------------------------|:-----------------------------|:----------------|:-------------|:-------------|:------------------|
| 391996    | 6.508813001668548,3.37740316890347 | 6.650969799999999,3.3450307     | 2021-07-01        | 2021-07-01 07:29:37 | Thursday      | 7             | 1              | July    | 6.50881                | 3.3774                  | 6.65097                     | 3

In [ ]:
df.head

In [17]:
completed_orders = pd.read_csv('../data/completed_orders_with_features.csv')
drivers_location = pd.read_csv('../data/drivers_location_during_request.csv')

In [19]:
api_key = "6a3c58c45a46fa944097a6ca00cb8f8d" 


In [24]:
import pandas as pd
import requests

# 1. Rename `Trip ID` to `order_id` in completed_orders
completed_orders = completed_orders.rename(columns={'Trip ID': 'order_id'})

# 2. Convert `order_id` to string in both DataFrames
completed_orders['order_id'] = completed_orders['order_id'].astype(str)
drivers_location['order_id'] = drivers_location['order_id'].astype(str)

# 3. Merge the DataFrames on `order_id` using a left join
merged_df = completed_orders.merge(drivers_location, on='order_id', how='left')

# 4. Convert the `Trip Start Time` and `Trip End Time` columns to datetime in `merged_df`
merged_df['Trip Start Time'] = pd.to_datetime(merged_df['Trip Start Time'])
merged_df['Trip End Time'] = pd.to_datetime(merged_df['Trip End Time'])

# 5. Drop the updated_at and created_at columns
merged_df = merged_df.drop(columns=['updated_at', 'created_at'])


In [23]:
merged_df.head(10)

,order_id,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,day_of_week,hour_of_day,day_of_month,month,Trip Origin_latitude,...,is_holiday,is_weekend,Trip Start Date,id,driver_id,driver_action,lat,lng,created_at,updated_at
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01,2021-07-01 07:29:37,Thursday,7,1,July,6.508813,...,0,0,2021-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01,2021-07-01 07:07:28,Thursday,6,1,July,6.431671,...,0,0,2021-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01,2021-07-01 07:02:23,Thursday,6,1,July,6.631679,...,0,0,2021-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01,2021-07-01 07:29:42,Thursday,7,1,July,6.572757,...,0,0,2021-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01,2021-07-01 09:34:36,Thursday,9,1,July,6.601042,...,0,0,2021-07-01,1.0,243828.0,accepted,6.602207,3.270465,NaN,NaN
5,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01,2021-07-01 09:34:36,Thursday,9,1,July,6.601042,...,0,0,2021-07-01,2.0,243588.0,rejected,6.592097,3.287445,NaN,NaN
6,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01,2021-07-01 09:34:36,Thursday,9,1,July,6.601042,...,0,0,2021-07-01,3.0,243830.0,rejected,6.596133,3.281784,NaN,NaN
7,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01,2021-07-01 09:34:36,Thursday,9,1,July,6.601042,...,0,0,2021-07-01,4.0,243539.0,rejected,6.596142,3.280526,NaN,NaN
8,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01,2021-07-01 09:34:36,Thursday,9,1,July,6.601042,...,0,0,2021-07-01,5.0,171653.0,rejected,6.609232,3.288800,NaN,NaN
9,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01,2021-07-01 09:34:36,Thursday,9,1,July,6.601042,...,0,0,2021-07-01,6.0,245662.0,rejected,6.593095,3.287759,NaN,NaN


### Get the weather

In [29]:

# 5. Extract `latitude` and `longitude` from `Trip Origin` and `Trip Destination` in `merged_df`
for col in ['Trip Origin', 'Trip Destination']:
    merged_df[[f'{col}_latitude', f'{col}_longitude']] = merged_df[col].str.split(',', expand=True).astype(float)

# 6. Filter to keep only rows where `driver_action` is 'accepted' and create a copy
merged_df_accepted_copy = merged_df[merged_df['driver_action'] == 'accepted'].copy()

# 7. Define a function `get_weather` that takes latitude, longitude, and API key as input and returns the weather description from the OpenWeatherMap API.
def get_weather(lat, lon, api_key):
    base_url = "https://api.openweathermap.org/data/3.0/onecall?"
    complete_url = base_url + "lat=" + str(lat) + "&lon=" + str(lon) + "&appid=" + api_key
    response = requests.get(complete_url)
    if response.status_code == 200:
        data = response.json()
        weather_description = data['current']['weather'][0]['description']
        return weather_description
    else:
        print("Error in API call")
        return None

# 8. Apply the `get_weather` function to the `merged_df_accepted_copy` DataFrame to get the weather conditions for each driver's location.
merged_df_accepted_copy['weather_description'] = merged_df_accepted_copy.apply(
    lambda row: get_weather(row['lat'], row['lng'], api_key), axis=1
)

# 9. Display the first 5 rows of the dataframe with the weather description.
print(merged_df_accepted_copy[['order_id', 'weather_description']].head().to_markdown(index=False, numalign="left", stralign="left"))

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x75bb2a7a65e0>>
Traceback (most recent call last):
  File "/home/hilla/code/10Academy-training/week8/Causal-Inference-Delivery-Logistic-Location-Optimization/.venv/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 

KeyboardInterrupt



In [28]:
merged_df_accepted_copy.head()

,order_id,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,day_of_week,hour_of_day,day_of_month,month,Trip Origin_latitude,...,is_holiday,is_weekend,Trip Start Date,id,driver_id,driver_action,lat,lng,created_at,updated_at
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01,2021-07-01 09:34:36,Thursday,9,1,July,6.601042,...,0,0,2021-07-01,1.0,243828.0,accepted,6.602207,3.270465,NaN,NaN
14,392005,"6.565087699999999,3.3844415","6.499696300000001,3.3509075",2021-07-01,2021-07-01 11:27:51,Thursday,10,1,July,6.565088,...,0,0,2021-07-01,11.0,245597.0,accepted,6.549147,3.392184,NaN,NaN
65,392009,"6.6636484,3.3082058","6.6185421,3.301634",2021-07-01,2021-07-01 07:41:12,Thursday,6,1,July,6.663648,...,0,0,2021-07-01,62.0,245600.0,accepted,6.644829,3.289328,NaN,NaN
132,392013,"6.4308171,3.4341552","6.435460000000001,3.4846547",2021-07-01,2021-07-01 09:19:11,Thursday,8,1,July,6.430817,...,0,0,2021-07-01,129.0,243892.0,accepted,6.435331,3.424317,NaN,NaN
145,392014,"6.499156300000001,3.3585173","6.4280911,3.5157172",2021-07-01,2021-07-01 07:27:24,Thursday,6,1,July,6.499156,...,0,0,2021-07-01,142.0,243781.0,accepted,6.498221,3.360042,NaN,NaN


In [31]:
df.to_csv('../data/features/merged_df_accepted_copy.csv', index=False)

In [25]:
# API call to request historical data:

lat, lon = (6.6010417,3.2766339)
time = 1627830000
api_key = os.getenv("OPENWEATHERMAP_API_KEY")

response = requests.get(
            f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={time}&appid={api_key}"
          )

def get_weather(timestamp, lat, lon):
    base_url = "http://api.openweathermap.org/data/2.0/onecall?"
    complete_url = base_url + "lat=" + str(lat) + "&lon=" + str(lon) + "&appid=" + api_key
    response = requests.get(complete_url) 
    x = response.json()
    if x["cod"] != "404":
        return x['weather'][0]['main']  # Simplify to just the main weather condition
    else:
        return np.nan  # Missing if API call fails

In [27]:
response.json()

{'lat': 6.601,
 'lon': 3.2766,
 'timezone': 'Africa/Lagos',
 'timezone_offset': 3600,
 'data': [{'dt': 1627830000,
   'sunrise': 1627796462,
   'sunset': 1627841116,
   'temp': 301.54,
   'feels_like': 303.45,
   'pressure': 1016,
   'humidity': 62,
   'dew_point': 293.56,
   'clouds': 40,
   'visibility': 10000,
   'wind_speed': 5.14,
   'wind_deg': 250,
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03d'}]}]}

In [10]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime

# Assuming you have your 'completed_orders' DataFrame loaded
# (including 'Trip Start Time', 'Trip Origin_latitude', and 'Trip Origin_longitude')

# 1. Public Holiday Data (Example using a public API)
# Get public holidays for Nigeria (replace with appropriate API/data source if needed)
def get_public_holidays(year):
    url = f"https://date.nager.at/api/v3/PublicHolidays/{year}/NG"
    response = requests.get(url)
    if response.status_code == 200:
        return [datetime.strptime(holiday['date'], '%Y-%m-%d') for holiday in response.json()]
    else:
        return []

holidays_2021 = get_public_holidays(2021)  # Adjust years as needed for your data

df['is_holiday'] = df['Trip Start Time'].dt.date.isin(holidays_2021).astype(int)

# 2. Weekend vs. Weekday
df['is_weekend'] = df['Trip Start Time'].dt.dayofweek.isin([5, 6]).astype(int)

# 3. Hour of the Day (Already extracted in previous steps)

# 4. Weather Data (Example using OpenWeatherMap API)
# You'll need to get an API key from OpenWeatherMap
api_key = "YOUR_API_KEY" 

def get_weather(timestamp, lat, lon):
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "lat=" + str(lat) + "&lon=" + str(lon) + "&appid=" + api_key
    response = requests.get(complete_url) 
    x = response.json()
    if x["cod"] != "404":
        return x['weather'][0]['main']  # Simplify to just the main weather condition
    else:
        return np.nan  # Missing if API call fails

# Apply the weather function to get weather conditions (This might take a while for a large dataset)
df['weather_condition'] = df.apply(lambda row: get_weather(row['Trip Start Time'], row['Trip Origin_latitude'], row['Trip Origin_longitude']), axis=1)

# 5. Traffic Data (Example using TomTom API or other traffic data sources)
# You'll need to adapt this to your chosen API/data source.
# This is a complex process and may require additional data processing based on your API's response.

# 6. Special Occasions (This requires manual research or finding relevant datasets)
# You'll need to identify special events in Lagos for your data's time period and create a feature.


# 7. Relevance Analysis
# Example: Use a logistic regression model to assess feature importance
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Choose features and target variable
X = df[['is_holiday', 'is_weekend', 'hour_of_day', 'weather_condition']]  # Add more features as available
y = df['trip_duration'] # Or another target variable representing order completion

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

# Print feature importance
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': model.coef_[0]})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
print(feature_importance)


ModuleNotFoundError: No module named 'requests'

In [16]:

# 4. Weather Data (Example using OpenWeatherMap API)
# You'll need to get an API key from OpenWeatherMap
api_key = "6a3c58c45a46fa944097a6ca00cb8f8d" 

def get_weather(timestamp, lat, lon):
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "lat=" + str(lat) + "&lon=" + str(lon) + "&appid=" + api_key
    response = requests.get(complete_url) 
    x = response.json()
    if x["cod"] != "404":
        return x['weather'][0]['main']  # Simplify to just the main weather condition
    else:
        return np.nan  # Missing if API call fails

# Apply the weather function to get weather conditions (This might take a while for a large dataset)
df['weather_condition'] = df.apply(lambda row: get_weather(row['Trip Start Time'], row['Trip Origin_latitude'], row['Trip Origin_longitude']), axis=1)


KeyError: 'weather'

In [6]:
import requests
import polyline
import pandas as pd
from geopy.distance import geodesic
from datetime import datetime

# Replace 'YOUR_API_KEY' with your actual OpenRouteService API key
ORS_API_KEY = '5b3ce3597851110001cf624897f57a95cf064c12bc59c0d460fdf85f'

def calculate_route_info(start_coords, end_coords, start_time, end_time):
    """Calculates route information using OpenRouteService API."""

    url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={ORS_API_KEY}&start={start_coords[1]},{start_coords[0]}&end={end_coords[1]},{end_coords[0]}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        # Extract key information from the API response
        total_distance = data['features'][0]['properties']['summary']['distance']
        total_duration = data['features'][0]['properties']['summary']['duration']  # In seconds
  # Corrected polyline decoding:
        decoded_polyline = polyline.decode(data['features'][0]['geometry']['coordinates'], geojson=True)
        
        # Calculate additional metrics
        straight_line_distance = geodesic(start_coords, end_coords).meters
        average_speed = (total_distance / total_duration) * 3.6  # Convert m/s to km/h
        trip_duration = end_time - start_time  # Assuming datetime objects

        return {
            'start_coords': start_coords,
            'end_coords': end_coords,
            'start_time': start_time,
            'end_time': end_time,
            'total_distance': total_distance,
            'total_duration': total_duration,
            'straight_line_distance': straight_line_distance,
            'average_speed': average_speed,
            'trip_duration': trip_duration.total_seconds(),
            'route_coordinates': decoded_polyline
        }
    else:
        print(f"Error: {response.status_code}")
        return None

# Example usage with the 'completed_orders' DataFrame
def enrich_completed_orders(df):
    results = []
    for _, row in df.iterrows():
        start_coords = (row['Trip Origin_latitude'], row['Trip Origin_longitude'])
        end_coords = (row['Trip Destination_latitude'], row['Trip Destination_longitude'])
        start_time = row['Trip Start Time']
        end_time = row['Trip End Time']
        
        route_info = calculate_route_info(start_coords, end_coords, start_time, end_time)
        if route_info:
            results.append(route_info)
    
    return pd.DataFrame(results)

# enrich your `completed_orders` DataFrame
enriched_df = enrich_completed_orders(df)
print(enriched_df.head().to_markdown(index=False, numalign="left", stralign="left"))


TypeError: ord() expected string of length 1, but list found